# Fit $G(t)$

\begin{equation}
G(t) = G_N^0 \int \frac{h\left(\tau\right)}{\tau} e^{-\frac{t}{\tau}} \mathrm{d}\tau \\
\end{equation}

If we don't know the form of relaxation spectrum a priori, we can start with the discrete spectrum
\begin{equation}
h\left(\tau\right) = \sum_i g_i \delta(\tau - \tau_i)
\end{equation}

\begin{equation}
G(t) = G_N^0 \int \frac{h\left(\tau\right)}{\tau} e^{-\frac{t}{\tau}} \mathrm{d}\tau = G_N^0 \sum_i \frac{g_i}{\tau_i} e^{-\frac{t}{\tau_i}} = \sum_i g_i^\prime e^{-\frac{t}{\tau_i}}
\end{equation}

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares, minimize
from scipy.misc import logsumexp

from mpmath import *
mp.dps = 25; mp.pretty = True
from scipy.optimize import broyden1

In [2]:
def heaviside(x):
    return 0.5*(np.sign(x)+1)

#Define BSW functions: G(t), G*(omega), h(tau)
def supp_prod(tauv, alpha, i):
    result = 1
    for j in range (1, i+1):
        result *= tauv[j]**(alpha[j - 1] - alpha[j])

    return result

def Gs_BSW(frequency, alpha, tauv, n, G0):
    result = []
    for omega in frequency: 
        sum1 = 0
        for i in range(0, n):
            sum1 += gamma(1 + alpha[i])*(tauv[i + 1]**(1 + alpha[i])*hyp2f1(1, 1 + alpha[i], 2 + alpha[i], -j*tauv[i + 1]*omega)/gamma(2 + alpha[i]) - tauv[i]**(1 + alpha[i])*hyp2f1(1, 1 + alpha[i], 2 + alpha[i], -j*tauv[i]*omega)/gamma(2 + alpha[i]))*supp_prod(tauv, alpha, i)
        sum2 = 0
        for i in range(0, n):
            sum2 += (tauv[i + 1]**alpha[i] - tauv[i]**alpha[i])/alpha[i]*supp_prod(tauv, alpha, i)
        res =j*omega*sum1/sum2*G0
        result.append((re(result),im(result)))

    return result

def Gt_BSW(time, alpha, tauv, n, G0):
    result = []
    for t in time: 
        sum1 = 0
        for i in range(0, n):
            sum1 += t**alpha[i] * ( gammainc(-alpha[i],t/tauv[i+1]) - gammainc(-alpha[i],t/tauv[i]) )*supp_prod(tauv, alpha, i)
        sum2 = 0
        for i in range(0, n):
            sum2 += (tauv[i + 1]**alpha[i] - tauv[i]**alpha[i])/alpha[i]*supp_prod(tauv, alpha, i)
        result.append(sum1/sum2*G0)

    return result

def h_BSW(time, alpha, tauv, n):
    result = []
    for t in time:
        sum1 = 0
        for i in range(0, n):
            sum1 += t**alpha[i] * heaviside(tauv[i+1]-t) * heaviside(t-tauv[i]) * supp_prod(tauv, alpha, i)
        sum2 = 0
        for i in range(0, n):
            sum2 += (tauv[i + 1]**alpha[i] - tauv[i]**alpha[i])/alpha[i]*supp_prod(tauv, alpha, i)
        result.append(sum1/sum2)

    return result

#Define BSW fits for linear chains based on paper 'Analytic slip-link expressions for universal dynamic modulus predictions of linear monodisperse polymer melts'
def Gs_CFSM(omega, Nc):
    n=3
    alpha1 = (-0.0005, -0.0205)
    alpha2 = (0.00029, 0.109957)
    alpha3 = (17.69589, 1.04026, -0.00095677)
    tau1 = (0.6288876, 0.119458)
    tau2 = (1.52508156, 0.02996758796795)
    tau3 = (3.110954, 0.022615)
    tau4 = (3.4840295, 0.0142809)
    alpha = (alpha1[0]*Nc + alpha1[1], alpha2[0]*Nc + alpha2[1], alpha3[0]/Nc + alpha3[1] + alpha3[2]*Nc)
    tauv = (tau1[1]*Nc**tau1[0], tau2[1]*Nc**tau2[0], tau3[1]*Nc**tau3[0], tau4[1]*Nc**tau4[0])
    return Gs_BSW(omega, alpha, tauv, n, 1)

def Gt_CFSM(t, Nc):
    n=3
    alpha1 = (-0.0005, -0.0205)
    alpha2 = (0.00029, 0.109957)
    alpha3 = (17.69589, 1.04026, -0.00095677)
    tau1 = (0.6288876, 0.119458)
    tau2 = (1.52508156, 0.02996758796795)
    tau3 = (3.110954, 0.022615)
    tau4 = (3.4840295, 0.0142809)
    alpha = (alpha1[0]*Nc + alpha1[1], alpha2[0]*Nc + alpha2[1], alpha3[0]/Nc + alpha3[1] + alpha3[2]*Nc)
    tauv = (tau1[1]*Nc**tau1[0], tau2[1]*Nc**tau2[0], tau3[1]*Nc**tau3[0], tau4[1]*Nc**tau4[0])
    return Gt_BSW(t, alpha, tauv, n, 1)

def h_CFSM(t, Nc):
    n=3
    alpha1 = (-0.0005, -0.0205)
    alpha2 = (0.00029, 0.109957)
    alpha3 = (17.69589, 1.04026, -0.00095677)
    tau1 = (0.6288876, 0.119458)
    tau2 = (1.52508156, 0.02996758796795)
    tau3 = (3.110954, 0.022615)
    tau4 = (3.4840295, 0.0142809)
    alpha = (alpha1[0]*Nc + alpha1[1], alpha2[0]*Nc + alpha2[1], alpha3[0]/Nc + alpha3[1] + alpha3[2]*Nc)
    tauv = (tau1[1]*Nc**tau1[0], tau2[1]*Nc**tau2[0], tau3[1]*Nc**tau3[0], tau4[1]*Nc**tau4[0])
    return h_BSW(t, alpha, tauv, n)

def Gs_R(omega):
    n=3
    alpha = (0.64635, -0.4959, -1.2716)
    tauv = (6.313268381616272e-9, 2.181509372282138e-7, 0.797317365925168,  18.201382525250114)
    GR = 1942.29;
    return Gs_BSW(omega, alpha, tauv, n, GR)

def Gt_MMM(time, params):
    #Variable frequencies
    lambdaArr = np.split(params,2)[0]
    gArr = np.split(params,2)[1]/np.sum(np.split(params,2)[1])
    #Fixed frequencies
    #lambdaArr=10.0**((np.array(range(nmodes), float) + 1.0)/nmodes*np.log10(tfinal))
    #gArr = params/np.sum(params)
    return np.dot(np.exp(-time/lambdaArr), gArr)*GN0

def log_Gt_MMM(time,params):
    lambdaArr = np.split(params,2)[0]
    gArr = np.split(params,2)[1]/np.sum(np.split(params,2)[1])
    #lambdaArr=10.0**((np.array(range(nmodes), float) + 1.0)/nmodes*np.log10(tfinal))
    #gArr = params/np.sum(params)
    return logsumexp(-time/lambdaArr, b=gArr*GN0)

#Vectorize function fdt and log_fdt
Gt_MMM_vec=np.vectorize(Gt_MMM, excluded=['params'])
log_Gt_MMM_vec=np.vectorize(log_Gt_MMM, excluded=['params'])

#Define residuals
def residuals_Gt_MMM(param):
    return Gt_MMM_vec(time=x, params=param)-y

def residuals_log_Gt_MMM(param):
    if np.any(Gt_MMM_vec(time=x[:-1], params=param) < 0):
        return np.full(x[:-1].shape,1e8) #Penalty for negative f_d(t)
    else:
        return log_Gt_MMM_vec(time=x, params=param)-np.log(y)

def MSE_MMM(param):
    return np.dot(residuals_Gt_MMM(param),residuals_Gt_MMM(param))/np.size(x)

def log_MSE_MMM(param):
    return np.dot(residuals_log_Gt_MMM(param),residuals_log_Gt_MMM(param))/np.size(x)

def Gp_MMM(omega, params):
    lambdaArr = np.split(params,2)[0]
    gArr = np.split(params,2)[1]/np.sum(np.split(params,2)[1])

    return np.sum((gArr * lambdaArr**2 * omega**2)/(1 + lambdaArr**2 * omega**2))*GN0

def Gdp_MMM(omega, params):
    lambdaArr = np.split(params,2)[0]
    gArr = np.split(params,2)[1]/np.sum(np.split(params,2)[1])

    return np.sum((gArr * lambdaArr * omega)/(1 + lambdaArr**2 * omega**2))*GN0

#Vectorize function Gp and Gdp
Gp_MMM_vec=np.vectorize(Gp_MMM, excluded=['params'])
Gdp_MMM_vec=np.vectorize(Gdp_MMM, excluded=['params'])

**First, read $G(t)$ from code**

In [11]:
with open('G.dat') as f:
    lines = f.readlines()
    x = np.array([float(line.split()[0]) for line in lines])
    y = np.array([float(line.split()[1]) for line in lines])

cutoff = min(np.argmax(y[1:]-y[:-1]>0), np.argmax(y<0))
x=x[:cutoff]
y=y[:cutoff]
tfinal=x[-1]
tstart=x[1]
GN0=y[0]

fig = plt.figure(figsize=(8, 6))

ax1 = fig.add_subplot(111)

ax1.set_title("Entanglement lifetime distribution")
ax1.set_xlabel(r'$t/\tau_c$')
ax1.set_ylabel(r'$G(t)$')
plt.ylim(ymin=y[-1], ymax=GN0)
plt.xlim(xmin=1, xmax=tfinal)
ax1.scatter(x,y, c='r', label=r'$G(t)$')

leg = ax1.legend()
ax1.set_xscale('log')
ax1.set_yscale('log')

plt.show()

## Optimizing strategy:
1. Start with some big number of modes, like $n_{modes}=10$
2. Run the least square optimization with standard residuals $y_i-f(x_i)$.
3. Scan through different number of modes down to 1.
4. If fit results have any negative weights, $g_i<0$, ignore
5. If fit results have all positive weights, run the least square optimization with log-residuals
6. Choose the best result among step 5

** Use SciPy numeric least square minimization algorithm **

## 1. Fit G(t) with Multi-Mode Maxwell expressions. Optimize using standard residuals $G_i-G^{MMM}(t_i)$. Choose $\lambda_i$ so they cover the whole range of data equidistantly on log-scale.Allow only $g_i$ to be adjusted

In [24]:
fits_1 = [] #output of fitting function for all tested numbers of modes
successful_fits_1 = [] #number of modes for successful fits
for nmodes in range(2, 15):
    lambdaArrInit=np.e**(np.log(tstart)+(np.array(range(nmodes), float))/(nmodes-1)*(np.log(tfinal)-np.log(tstart)))
    fit = np.linalg.lstsq(np.exp(-np.outer(x,1.0/lambdaArrInit)), y)[0]
    fits_1.append(fit)
    #if not np.any(Gt_MMM_vec(time=x, params=np.append(lambdaArrInit, fit)) < 0):
    if not np.any(fit < 0):
        successful_fits_1.append(nmodes)
        print nmodes, fit, MSE_MMM(np.append(lambdaArrInit, fit))

In [38]:
nmodes=8
lambdaArrInit=np.e**(np.log(tstart)+(np.array(range(nmodes), float))/(nmodes-1)*(np.log(tfinal)-np.log(tstart)))
fit = fits_1[nmodes-2]
fig1 = plt.figure(figsize=(24, 6))

ax0 = fig1.add_subplot(131)

ax0.set_title(r'$h\left(\tau\right)/\tau$')
ax0.set_xlabel(r'$\lambda$')
ax0.set_ylabel(r'$g$')

ax0.scatter(lambdaArrInit, fit/np.sum(fit), c='r', label=r'First fit')
leg = ax0.legend()
ax0.set_xscale('log')

ax1 = fig1.add_subplot(132)

ax1.set_title("Check results of the fit")
ax1.set_xlabel(r'$t/\tau_c$')
ax1.set_ylabel(r'log residuals')

ax1.plot(x,Gt_MMM_vec(time=x, params=np.append(lambdaArrInit,fit)), c='r', label=r'First fit')
ax1.plot(x,y, c='g', label=r'$f_d(t)$')

leg = ax1.legend()
ax1.set_xscale('log')
ax1.set_yscale('log')

# ax2 = fig1.add_subplot(133)

# ax2.set_title(r'$f_d(t)$')
# ax2.set_xlabel(r'$t/\tau_c$')
# ax2.set_ylabel(r'$f_d(t)$')

# ax2.plot(x,Gt_MMM_vec(time=x, params=fit.x), c='r', label=r'First fit')
# ax2.plot(x,y, c='g', label=r'Simulation data')
# leg = ax2.legend()
# ax2.set_xscale('log')

plt.show()

## 2. Improve fit using log-residuals $\log(G_i)-\log(G^{MMM}(t_i))$ to find best fit for the longest relaxation tail 

In [55]:
fits_2 = [] #output of fitting function for all tested numbers of modes
min_log_SME = log_MSE_MMM(fits_1[successful_fits_1[0]-2])
best_nmodes = successful_fits_1[0]

for i in successful_fits_1:
    fit = fits_1[i-2]
    nmodes = i
    lambdaArrInit=np.e**(np.log(tstart)+(np.array(range(nmodes), float))/(nmodes-1)*(np.log(tfinal)-np.log(tstart)))
    print('nmodes\t{0}'.format(i))

    fit2 = least_squares(residuals_log_Gt_MMM, np.append(lambdaArrInit, fit), xtol=1e-14, ftol=1e-14)
    fits_2.append(fit2)

    print('First fit log-MSE\t{0}'.format(log_MSE_MMM(np.append(lambdaArrInit, fit))))

    if fit2.success:
        weights = np.split(fit2.x, 2)[1]/np.sum(np.split(fit2.x, 2)[1])
        if log_MSE_MMM(fit2.x)<min_log_SME and not np.any(weights<0):
            min_log_SME = log_MSE_MMM(fit2.x)
            best_fit = fit2
            best_nmodes = i
        print('Second fit log-MSE\t{0}'.format(log_MSE_MMM(fit2.x)))
        print(fit2.message)
        print('Weights\t{0}'.format(weights))

    print(' ')

In [56]:
best_nmodes

In [58]:
best_fit.x

In [74]:
nmodes = best_nmodes
lambdaArrInit=np.e**(np.log(tstart)+(np.array(range(nmodes), float))/(nmodes-1)*(np.log(tfinal)-np.log(tstart)))
fit = fits_1[best_nmodes-2]
fit2 = best_fit

fig3 = plt.figure(figsize=(24, 6))

ax0 = fig3.add_subplot(131)

ax0.set_title(r'$p^{cr}\left(\tau\right)$')
ax0.set_xlabel(r'$\lambda$')
ax0.set_ylabel(r'$g$')

ax0.scatter(lambdaArrInit, fit/np.sum(fit), c='r', label=r'First fit')
ax0.scatter(np.split(fit2.x,2)[0],np.split(fit2.x,2)[1]/np.sum(np.split(fit2.x,2)[1]), c='b', label=r'Second fit')
leg = ax0.legend()
ax0.set_xscale('log')
ax0.set_yscale('log')

ax1 = fig3.add_subplot(132)

ax1.set_title("Check results of the fit")
ax1.set_xlabel(r'$t/\tau_c$')
ax1.set_ylabel(r'log residuals')

ax1.plot(x,Gt_MMM_vec(time=x, params=np.append(lambdaArrInit,fit)), c='r', label=r'First fit')
ax1.plot(x,Gt_MMM_vec(time=x, params=fit2.x), c='b', label=r'Second fit')
ax1.plot(x,y, c='g', label=r'$f_d(t)$')

leg = ax1.legend()
ax1.set_xscale('log')
ax1.set_yscale('log')

ax2 = fig3.add_subplot(133)

ax2.set_title(r'$f_d(t)$')
ax2.set_xlabel(r'$t/\tau_c$')
ax2.set_ylabel(r'$f_d(t)$')

ax2.plot(x,Gt_MMM_vec(time=x, params=np.append(lambdaArrInit, fit)), c='r', label=r'First fit')
ax2.plot(x,Gt_MMM_vec(time=x, params=fit2.x), c='b', label=r'Second fit')
ax2.plot(x,y, c='g', label=r'Simulation data')
leg = ax2.legend()
ax2.set_xscale('log')

plt.show()

In [76]:
omegaPoints = 1000
omegaMin = -8
omegaMax = 5
omegaArr=10**(omegaMin+(np.array(range(omegaPoints), float) + 1.0)/omegaPoints*(omegaMax-omegaMin))

fig0 = plt.figure(figsize=(8, 6))
ax0 = fig0.add_subplot(111)
ax0.set_title(r'Dynamic modulus $G*(\omega)$')
ax0.set_xlabel(r'$\omega$')
ax0.set_ylabel(r'$G*$')

ax0.plot(omegaArr,Gp_MMM_vec(omega=omegaArr,params=fit2.x), c='k', label=r'$G^\prime$')
ax0.plot(omegaArr,Gdp_MMM_vec(omega=omegaArr,params=fit2.x), c='k', label=r'$G^{\prime\prime}$')

leg = ax0.legend()
ax0.set_yscale('log')
ax0.set_xscale('log')
plt.show()